In [181]:
import pandas as pd
pd.set_option('display.max_rows', None)  # 显示所有行
pd.set_option('display.max_columns', None)  # 显示所有列

In [182]:
import pandas as pd

# 读入 CSV 文件
df = pd.read_csv('more_layer.csv')
# df = pd.read_csv('evaluate_seg.csv')

# df = df.drop(columns=['pbid'])
# df = df[df['ARG'] <= 100000]
# df = df[df['classcial'] <= 500]
# df = df[df['in_constraints_probs'] >= 99]
grouped_df = df.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
    'iteration_count':'mean',
    'classcial':'mean',
    'run_times':'mean',
    "ARG": 'mean',
    'in_constraints_probs': 'mean',
    'best_solution_probs': 'mean',
})


## 分组并把组作为索引
pivot_df = grouped_df.pivot(index =['pkid', 'layers', 'variables','constraints'], columns='method', values=["ARG", 'best_solution_probs', 'classcial', 'run_times','iteration_count'])
# pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])

method_order = ['HeaSolver', 'PenaltySolver', 'ChocoSolver']
# method_order = ['HeaSolver', 'PenaltySolver', 'ChocoSolver', 'QtoSolver ', 'QtoSimplifySolver', 'QtoSimplifyDiscardSolver', 'QtoSimplifyDiscardSegmentedSolver']
# method_order = ['ChocoSolver', 'QtoSolver', 'QtoSimplifySolver', 'QtoSimplifyDiscardSolver']
# method_order = ['ChocoSolver']
# method_order = ['QtoSimplifyDiscardSegmentedSolver']
pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([["ARG",'best_solution_probs','run_times','iteration_count', 'classcial'], method_order]))

pivot_df

ARG                            \
                                   HeaSolver PenaltySolver ChocoSolver   
pkid layers variables constraints                                        
0    1      6         3             1.277500    122.380007    0.366299   
     2      6         3            47.624587     86.587841    0.100954   
     3      6         3            89.135591     77.521218    0.179157   
     4      6         3            90.667043    100.612672    0.069101   
     5      6         3            76.610041     86.307962    0.068237   
     6      6         3            76.980695     71.865827    0.063459   
     7      6         3            70.842196     81.226043    0.048690   
     8      6         3                  NaN           NaN    0.034185   

                                  best_solution_probs                \
                                            HeaSolver PenaltySolver   
pkid layers variables constraints                                     
0    1      6         3                      0.000000      2.811523   
     2      6         3                      0.474609      5.166016   
     3      6         3                      2.565430      7.884766   
     4      6         3                      4.608398      3.502930   
     5      6         3                      4.549805      6.520508   
     6      6         3                      4.826172      7.627930   
     7      6         3                      7.223633      6.868164   
     8      6         3                           NaN           NaN   

                                                run_times                \
                                  ChocoSolver   HeaSolver PenaltySolver   
pkid layers variables constraints                                         
0    1      6         3             49.973633   213329.92      33658.88   
     2      6         3             77.992188   470487.04      55511.04   
     3      6         3             80.924805   604272.64      72755.20   
     4      6         3             90.208984   775454.72      95457.28   
     5      6         3             91.155273   999198.72     117841.92   
     6      6         3             90.792969  1020416.00     141752.32   
     7      6         3             93.075195  1025024.00     160757.76   
     8      6         3             94.733398         NaN           NaN   

                                              iteration_count                \
                                  ChocoSolver       HeaSolver PenaltySolver   
pkid layers variables constraints                                             
0    1      6         3              29409.28          207.33         31.87   
     2      6         3              50135.04          458.46         53.21   
     3      6         3              73666.56          589.11         70.05   
     4      6         3              94556.16          756.28         92.22   
     5      6         3             112250.88          975.35        114.08   
     6      6         3             129075.20          996.39        137.43   
     7      6         3             151511.04         1001.00        155.99   
     8      6         3             170967.04             NaN           NaN   

                                                classcial                \
                                  ChocoSolver   HeaSolver PenaltySolver   
pkid layers variables constraints                                         
0    1      6         3                 27.72   18.310068     10.375388   
     2      6         3                 47.96  138.279568     16.956953   
     3      6         3                 70.94  410.508151     42.161196   
     4      6         3                 91.34  375.470330     76.091785   
     5      6         3                108.62  327.111023     27.702380   
     6      6         3                125.05  262.618317     33.350603   
     7      6         3                146.96  419.188909    157.325145  

In [183]:
a = pivot_df[('classcial', 'ChocoSolver')].sum()
b = pivot_df[('classcial', 'QtoSimplifyDiscardSolver')].sum()
c = pivot_df[('classcial', 'ChocoSolver')].mean()
d = pivot_df[('classcial', 'QtoSimplifyDiscardSolver')].mean()
print(f'choco:{c}, qtodiscard:{d}')
print(d/c)


KeyError: ('classcial', 'QtoSimplifyDiscardSolver')

In [ ]:
# # 假设“variables”和“constraints”列作为多重索引列的一部分
# # 使用 .loc 选择多重索引列中含“variables”和“constraints”相关的列
# selected_columns = pivot_df.loc[:, pivot_df.columns.get_level_values(0).isin(['variables', 'constraints'])]

# selected_columns


In [ ]:
# 选取 'depth' 下的所有列，并剔除 'QtoSolver ' 和 'QtoSimplifySolver'
df_1 = pivot_df['ARG'].drop(columns=['QtoSolver ', 'QtoSimplifySolver'], errors='ignore')

# 选取 'culled_depth_per_param' 下的 'QtoSimplifyDiscardSolver' 列
df_2 = pivot_df['classcial'].drop(columns=['QtoSolver ', 'QtoSimplifySolver'], errors='ignore')

# 合并处理后的 'depth' 列与 'QtoSimplifyDiscardSolver' 列，并指定后缀避免冲突
culled_depth_df = df_1.join(df_2, lsuffix='_1', rsuffix='_2')

# 按行遍历输出值，每个值用 & 分隔
for row in culled_depth_df.itertuples(index=False):
    formatted_row = [f"{round(x, 3):6.03f}" for x in row]
    print(" & ".join(formatted_row))


   nan &    nan &    nan &    nan


In [ ]:
# # 从 pivot_df 中提取 variables 和 constraints 列
# variables_constraints_df = pivot_df.index.to_frame(index=False)[['variables', 'constraints']]
# variables_constraints_df

In [ ]:
# 提取所需列
best_solution_probs_Qto_discard = pivot_df['best_solution_probs']['QtoSimplifyDiscardSolver']
best_solution_probs_choco = pivot_df['best_solution_probs']['ChocoSolver']
arg_Qto_discard = pivot_df['ARG']['QtoSimplifyDiscardSolver']
arg_choco = pivot_df['ARG']['ChocoSolver']

# 方法1：每行相除取平均
# best_solution_probs 下 QtoSimplifyDiscardSolver / ChocoSolver 的倍数
row_wise_ratio_best_solution_probs = (best_solution_probs_Qto_discard / best_solution_probs_choco).mean()

# ARG 下 ChocoSolver / QtoSimplifyDiscardSolver 的倍数
row_wise_ratio_arg = (arg_choco / arg_Qto_discard).mean()

# 方法2：整列取平均后相除
# best_solution_probs 下 QtoSimplifyDiscardSolver / ChocoSolver 的整列平均的倍数
col_avg_ratio_best_solution_probs = best_solution_probs_Qto_discard.mean() / best_solution_probs_choco.mean()

# ARG 下 ChocoSolver / QtoSimplifyDiscardSolver 的整列平均的倍数
col_avg_ratio_arg = arg_choco.mean() / arg_Qto_discard.mean()

# 输出结果
print(f"行平均 - QtoSimplifyDiscardSolver / ChocoSolver (best_solution_probs): {row_wise_ratio_best_solution_probs:.2f}")
print(f"行平均 - ChocoSolver / QtoSimplifyDiscardSolver (ARG): {row_wise_ratio_arg:.2f}")
print(f"列平均 - QtoSimplifyDiscardSolver / ChocoSolver (best_solution_probs): {col_avg_ratio_best_solution_probs:.2f}")
print(f"列平均 - ChocoSolver / QtoSimplifyDiscardSolver (ARG): {col_avg_ratio_arg:.2f}")


行平均 - QtoSimplifyDiscardSolver / ChocoSolver (best_solution_probs): nan
行平均 - ChocoSolver / QtoSimplifyDiscardSolver (ARG): nan
列平均 - QtoSimplifyDiscardSolver / ChocoSolver (best_solution_probs): nan
列平均 - ChocoSolver / QtoSimplifyDiscardSolver (ARG): nan


In [ ]:

# # 读入 CSV 文件
# df = pd.read_csv('evaluate_TSP.csv')

# df = df.drop(columns=['pbid'])

# grouped_df = df.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
#     "ARG": 'mean',
#     'in_constraints_probs': 'mean',
#     'best_solution_probs': 'mean',
#     'iteration_count':'mean'
# })

# ## 分组并把组作为索引
# pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])

# method_order = ['PenaltySolver', 'CyclicSolver', 'HeaSolver', 'ChocoSolver']
# pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

# method_order = ['PenaltySolver', 'CyclicSolver', 'HeaSolver', 'ChocoSolver']
# pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

# pivot_df

In [ ]:
# import pandas as pd

# # 读入 CSV 文件
# df = pd.read_csv('eva_SCP.csv')

# df = df.drop(columns=['pbid'])

# grouped_df = df.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
#     "ARG": 'mean',
#     'in_constraints_probs': 'mean',
#     'best_solution_probs': 'mean',
#     'iteration_count':'mean'
# })

# ## 分组并把组作为索引
# pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])

# method_order = ['PenaltySolver', 'CyclicSolver', 'HeaSolver', 'ChocoSolver']
# pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

# pivot_df

In [ ]:
# 定义格式化函数
import numpy as np
def format_value(val):
    if np.isnan(val):  # 如果是NaN值则返回空字符
        return "NaN"
    elif val < 100:  # 如果值小于100，保留3位有效数字
        return f"{val:.2f}"
    else:  # 如果值大于等于100，不保留小数点
        return f"{val:.0f}"

# 按行遍历输出，每个值用 & 分隔
for row in pivot_df.itertuples(index=False):
    formatted_row = [format_value(x) for x in row]
    print(" & ".join(formatted_row))

NaN & NaN & NaN & NaN & NaN & NaN & NaN & NaN & NaN & NaN & NaN & NaN & NaN & NaN & NaN


In [ ]:
# import pandas as pd

# # 读入 CSV 文件
# df = pd.read_csv('eva_old.csv')

# df = df.drop(columns=['pbid'])

# grouped_df = df.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
#     "ARG": 'mean',
#     'in_constraints_probs': 'mean',
#     'best_solution_probs': 'mean',
#     'iteration_count':'mean'
# })
# grouped_df
# # 假设grouped_df是一个Pandas DataFrame
# grouped_df = grouped_df[((grouped_df['layers'] == 1) & (grouped_df['method'] == 'commute')) | ((grouped_df['layers'] == 7) & (grouped_df['method'] != 'commute'))]
# grouped_df = grouped_df.drop(columns=['layers'])
# ## 分组并把组作为索引
# pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])




# method_order = ['penalty', 'cyclic', 'HEA', 'commute']
# pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

# pivot_df

In [ ]:
pivot_df['ARG']

,,,,ChocoSolver,QtoSimplifySolver,QtoSimplifyDiscardSolver
pkid,layers,variables,constraints,,,
0,0,6,3,NaN,NaN,NaN


In [ ]:
# import pandas as pd

# # 读入 CSV 文件
# df = pd.read_csv('eval_eliminate copy 2.csv')

# df = df.drop(columns=['pbid','layers', 'variables', 'constraints'])

# gr    ouped_df = df.groupby(['pkid',  'method'], as_index=False).agg({
#     "successrate": 'mean',
# })
# grouped_df
# # 假设grouped_df是一个Pandas DataFrame
# # grouped_df = grouped_df[((grouped_df['layers'] == 1) & (grouped_df['method'] == 'commute')) | ((grouped_df['layers'] == 7) & (grouped_df['method'] != 'commute'))]
# # grouped_df = grouped_df.drop(columns=['layers'])
# ## 分组并把组作为索引
# pivot_df = grouped_df.pivot(index =['pkid'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])




# method_order = ['penalty', 'cyclic', 'HEA', 'commute']
# pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

# pivot_df